In [ ]:
# 1️⃣ 구글 드라이브 마운트

from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/MyDrive/likelion/project2/

Mounted at /gdrive
/gdrive/MyDrive/likelion/project2


In [ ]:
!pip install -U langchain-community
!pip install faiss-gpu
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver

In [ ]:
import json
import os
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from google.colab import drive
from google.colab import userdata


# 1️⃣ open_api_key 입력
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')


# 2️⃣ JSON 폴더 경로 지정
json_folder_path = './project_2/data/Other/Other/total'  # 📁 이 경로에 여러 json 파일이 있다고 가정

# 3️⃣ 폴더 내 모든 JSON 파일 탐색 및 로드
all_data = []
for filename in os.listdir(json_folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(json_folder_path, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # 파일이 리스트 형태면 확장, dict면 감싸서 처리
                if isinstance(data, list):
                    all_data.extend(data)
                elif isinstance(data, dict):
                    all_data.append(data)
            print(f"📂 Loaded: {filename}")
        except Exception as e:
            print(f"⚠️ Error loading {filename}: {e}")

print(f"✅ 총 {len(all_data)}개의 JSON 데이터 로드 완료!\n")

📂 Loaded: SL09-00-09-01.json
📂 Loaded: SL09-00-09-02.json
📂 Loaded: SL09-00-08-01.json
📂 Loaded: SL09-00-07-01.json
📂 Loaded: SL09-00-10-01.json
📂 Loaded: SL09-00-11-01.json
📂 Loaded: SL09-00-12-01.json
📂 Loaded: SL09-00-06-01.json
📂 Loaded: SL09-00-06-02.json
📂 Loaded: SL09-00-07-02.json
📂 Loaded: SL09-00-11-02.json
📂 Loaded: SL09-00-12-02.json
📂 Loaded: SL09-00-10-02.json
📂 Loaded: SL09-00-08-02.json
📂 Loaded: SL10-00-00-02.json
📂 Loaded: SL10-00-02-02.json
📂 Loaded: SL10-00-01-01.json
📂 Loaded: SL10-00-02-01.json
📂 Loaded: SL10-00-01-02.json
📂 Loaded: SL10-00-00-01.json
📂 Loaded: SL11-00-00-00.json
📂 Loaded: SL11-00-06-01.json
📂 Loaded: SL11-00-03-02.json
📂 Loaded: SL11-00-01-02.json
📂 Loaded: SL11-00-04-00.json
📂 Loaded: SL11-00-05-02.json
📂 Loaded: SL11-00-05-01.json
📂 Loaded: SL11-00-01-01.json
📂 Loaded: SL11-00-02-01.json
📂 Loaded: SL11-00-06-02.json
📂 Loaded: SL11-00-02-02.json
📂 Loaded: SL11-00-03-01.json
📂 Loaded: SL12-00-15-01.json
📂 Loaded: SL12-00-01-02.json
📂 Loaded: SL12

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # ✅ 변경
from langchain.docstore.document import Document
import os

# 1️⃣ 임베딩 모델 설정 (OpenAI 대신 Sentence-Transformers)
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask", model_kwargs={"device": "cpu"}, encode_kwargs={"normalize_embeddings": True})

# 2️⃣ 문서 리스트 생성
docs = []
for item in all_data:
    title = item.get("data_title", "")
    source = item.get("data_source", "")
    for content in item.get("contents", []):
        text = content.get("text", "")
        if not text.strip():
            continue
        meta = {
            "title": title,
            "source": source,
            "year": item.get("data_year", ""),
            "institution": item.get("data_institution", "")
        }
        docs.append(Document(page_content=text, metadata=meta))

print(f"🧾 총 {len(docs)}개의 문서(Document) 객체 생성 완료!\n")

# 3️⃣ FAISS 벡터스토어 초기화
faiss_index = None
faiss_save_path = './project_2/data/db'

# 4️⃣ 배치 단위로 임베딩 & 저장
batch_size = 500  # 한 번에 처리할 문서 개수
# for i in range(0, min(len(docs), 5000), batch_size):
for i in range(5000, len(docs), batch_size):
    batch_docs = docs[i:i+batch_size]

    if faiss_index is None:
        # 첫 배치이면 새로 생성
        faiss_index = FAISS.from_documents(batch_docs, embeddings)
    else:
        # 두 번째 배치부터는 추가
        faiss_index.add_documents(batch_docs)

    # 중간 저장
    os.makedirs(faiss_save_path, exist_ok=True)
    faiss_index.save_local(faiss_save_path)
    print(f"✅ {i + len(batch_docs)} / {len(docs)} 문서 임베딩 완료, FAISS 저장 완료")

print(f"\n🎉 모든 문서 임베딩 완료 및 FAISS DB 저장 완료!\n📍 저장 위치: {faiss_save_path}")


🧾 총 819058개의 문서(Document) 객체 생성 완료!

✅ 5500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 6000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 6500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 7000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 7500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 8000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 8500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 9000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 9500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 10000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 10500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 11000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 11500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 12000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 12500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 13000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 13500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 14000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 14500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 15000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 15500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 16000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 16500 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 17000 / 819058 문서 임베딩 완료, FAISS 저장 완료
✅ 17500 / 81

In [ ]:
# 8️⃣ Q&A CSV로 저장
qa_list = []
for item in all_data:
    for content in item.get("contents", []):
        for qa in content.get("QA", []):
            question = qa.get("question", "").replace("\n", " ").strip()
            answer = qa.get("answer", "").replace("\n", " ").strip()
            if question and answer:
                qa_list.append({"question": question, "answer": answer})

qa_df = pd.DataFrame(qa_list)
csv_save_path = './project_2/data/db_csv/rag_qa.csv'
qa_df.to_csv(csv_save_path, index=False, encoding='utf-8-sig')
print(f"✅ Q&A CSV 저장 완료!\n📍 저장 위치: {csv_save_path}")

✅ Q&A CSV 저장 완료!
📍 저장 위치: ./project_2/data/db_csv/rag_qa.csv
